In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

In [ ]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
print(text[:200])

In [ ]:
chars = sorted(list(set(text)))
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

In [ ]:
block_size = 8
n_emb = 32
vocab_size = len(chars)
batch_size = 32
eval_iters = 200
eval_interval = 5000
data = torch.tensor(encode(text), dtype=torch.long)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n = int(0.9*len(data))

In [ ]:
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size)
        
        
    def forward(self, idx, targets=None):
        B , T = idx.shape
        token_emb = self.token_embedding_table(idx) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = token_emb + pos_emb  #(B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B , T , C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits , loss

      
    def generate(self, context, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(context)
            logits = logits[:,-1,:] #focus on only the last time step
            probs = F.softmax(logits, dim=1)
            idx = torch.multinomial(probs, num_samples=1)
            context = torch.cat((context,idx), dim=1) #append new sampled index to the running context
        return context

In [ ]:
model = BigramLanguageModel()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

In [ ]:
max_iter = 10
for iter in range(max_iter):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb , yb = get_batch('train')
    logit , loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=100)[0].tolist()))

# saving and loading for inference 

In [ ]:
# torch.save(model.state_dict(), PATH)

# # loading

# model = BigramLanguageModel() #instantiate model arcitecture
# model.load_state_dict(torch.load(PATH))

# saving and loading to continue training

In [ ]:
# # saving
# torch.save({'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict()}, PATH)

# # loading
# checkpoint = torch.load(PATH, map_location=device)
# model = BigramLanguageModel() #instantiate model arcitecture
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3) #instantiate optimizer
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])